In [1]:
#learn kung fu
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mason_functions as mf

import os
import evaluate
import warnings
warnings.filterwarnings('ignore')

import sklearn.preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectKBest, f_regression, RFE

import wrangle

ModuleNotFoundError: No module named 'wrangle'

In [ ]:
#define my sql query to access the relational database
sql = '''
SELECT bathroomcnt, bedroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt
FROM predictions_2017
LEFT JOIN properties_2017 USING(parcelid)
WHERE propertylandusetypeid = 261 or propertylandusetypeid = 279
'''

#define my url
url = mf.get_db_url('zillow')

#read the information from a database into a dataframe
#I also will not continuously query the database; instead, I will write the data to a .csv (in case the kernel gotta go)
if os.path.isfile('zillow_nadir.csv'):
    df = pd.read_csv('zillow_nadir.csv', index_col = 0)
else:
    df = pd.read_sql(sql, url)
    df.to_csv('zillow_nadir.csv')

In [ ]:
#at a glance
pd.options.display.max_columns = 69
pd.options.display.max_rows = 69
df.info()

In [ ]:
#double-take
df.head()

In [ ]:
#ok, this hurts to look at. rename the columns
df = df.rename(columns = {'bathroomcnt': 'bathroom_count',
                  'bedroomcnt': 'bedroom_count',
                  'calculatedfinishedsquarefeet': 'square_footage',
                  'taxvaluedollarcnt': 'tax_value'
                  })

In [ ]:
#summary statistics
df.describe().T

In [ ]:
#Nulls?
df.isnull().any()

In [ ]:
#how many nulls and where?
df.isnull().sum()

In [ ]:
#quick reminder on total number observations
mf.add_commas(len(df))

In [ ]:
#okay, yeah, I do not need these 82 observations out of 50K total
#I will drop the row where the target value is null, because I cannot fathom a use for it at the moment
df = df[df.square_footage.notnull()]
df = df[df.tax_value.notnull()]

In [ ]:
#confirm dropped rows
mf.add_commas(len(df))

In [ ]:
#let's take another look
df.info()

In [ ]:
#data is prepped. Let's make a function
def acq_zillow_nadir():
        
    '''
    This function reads the data from a database into a dataframe.
    I also will not continuously query the database; instead, I will write the data to a .csv (in case the kernel gotta go)
    '''
    
    #define my sql query
    sql = '''
    SELECT bathroomcnt, bedroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt
    FROM predictions_2017
    LEFT JOIN properties_2017 USING(parcelid)
    WHERE propertylandusetypeid = 261 or propertylandusetypeid = 279
    '''

    #define my url using a url-retrieval function from my personal .py
    url = mf.get_db_url('zillow')

    #set up an if-conditional to see if there is a .csv readily available
    if os.path.isfile('zillow_nadir.csv'):
        
        #if there is, render a workable dataframe from the .csv
        df = pd.read_csv('zillow_nadir.csv', index_col = 0)
        
    #if not, access the relational database, and then write data to a .csv for later ease of access
    else:
        df = pd.read_sql(sql, url)
        df.to_csv('zillow_nadir.csv')
    
    #return the dataframe
    return df

In [ ]:
#let's do it again
def clean_zillow_nadir():
    '''
    This function acquires my zillow data, drops any observations where the values are missing, and renames the columns so that they are easier to work with.
    '''
    zillow_nadir = acq_zillow_nadir()
    zillow_nadir = zillow_nadir.rename(columns = {'bathroomcnt': 'bathroom_count',
                  'bedroomcnt': 'bedroom_count',
                  'calculatedfinishedsquarefeet': 'square_footage',
                  'taxvaluedollarcnt': 'tax_value'
                  })
    zillow_nadir = zillow_nadir[zillow_nadir.square_footage.notnull()]
    zillow_nadir = zillow_nadir[zillow_nadir.tax_value.notnull()]
    
    return zillow_nadir

In [ ]:
#one more time
def prep_zillow_nadir():
    #clean the dataframe
    zillow_nadir = clean_zillow_nadir()
    
    #split the data with a function from my personal .py
    train, validate, test = mf.split_data(zillow_nadir)
    
    #return the data
    return train, validate, test

In [ ]:
#test wrangle
train, validate, test = prep_zillow_nadir()

In [ ]:
#check shapes - looks clear
train.shape, validate.shape, test.shape